# Projet 08 : Réalisez un dashboard et assurez une veille technique :

In [2]:
import pandas as pd
import numpy as np
import pprint
import streamlit as st
import os
from IPython.display import display

## Etape 1 - Elaborez le Dashboard :

### 1.1 - Collecte d'informations :

**Contexte :**  
Suite à la réalisation de l'outil de scoring disponible sur cloud, on nous demande de créer une interface plus compète sous forme de Dashboard intéractif.

**Spécifications demandées :**  
* Visualisation du score et de la probalité, interprétable de façon intelligible pour un novice.
* Visualisation des principales informations descriptives du client traité.
* Graphiques de comparaison des principales informations descriptives par rapport à:
    * la totalité des clients
    * un groupe (filtres des infos descriptives)
* Graphiques couvrant des critères d'accessibilité du CWAG
* Dashboard déployé sur le cloud
* Optionnel :
    * Possibilité de mise à jour sur modification d'infos du client
    * Possibilité de changer de dossier client

---
### 1.2 Blueprint :

In [3]:
blueprint = pd.DataFrame({
    "Besoins": ["Détails du crédit","Visualisation du score et sa probabilité \n(loin du seuil ?)", "Interprétation du score de façon simple", "Principales informations descriptives du client","Comparaison des infos du client par rapport à: \n-L'ensemble des clients \nUn groupe (filtres)"  ],
    "Questions types": ["Quels sont les détails du crédit demandé ?", "Quel est le score obtenu par le client ? \nQuelle est la probabilité et où se trouve t-elle par rapport au seuil ?", "Quels paramètres ont contribués le plus au choix ?", "-Age, sexe, Possède une voiture ?, Possède un logement ? \n-Quelle est la situation professionnelle du client ? \n-Quelle est sa situation maritale ? enfant ?", "Où se trouve ce client par rapport aux autres ?"],
    "Indicateurs et données nécessaires": ["Données nécessaires: \n-Type de crédit \n-Montant du crédit \n\nIndicateurs à afficher : \n-NAME_CONTRACT_TYPE \n-AMT_CREDIT ", "Données nécessaires: \n-score client \n-probabilité client \n\nIndicateurs à afficher : \n-Business_score \n-predict_proba" , "Données nécessaires : \nFeature Importance Locale \n\nIndicateurs à afficher : \n-TreeExplainer de shap", "Données nécessaires : \n-Informations descriptives du client en cours \n\nIndicateurs à afficher : \n-SK_ID_CURR, DAYS_BIRTH, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY \n-AMT_INCOME_TOTAL \n-NAME_FAMILY_STATUS \n-CNT_CHILDREN", "Données nécessaires : informations descriptives de tous les clients \n\nIndicateurs à afficher : \n-Moyenne globale ou par groupe des informations descriptives"],
    "Visualisations": ["", "","", "", ""]
})

In [4]:
styled_blueprint = blueprint.style.set_properties(**{'white-space': 'pre-wrap', 'text-align': 'left'}).set_table_styles([
                                      {'selector': 'th', 'props': [('text-align', 'left')]}])
display(styled_blueprint)

,Besoins,Questions types,Indicateurs et données nécessaires,Visualisations
0,Détails du crédit,Quels sont les détails du crédit demandé ?,Données nécessaires: -Type de crédit -Montant du crédit Indicateurs à afficher : -NAME_CONTRACT_TYPE -AMT_CREDIT,
1,Visualisation du score et sa probabilité (loin du seuil ?),Quel est le score obtenu par le client ? Quelle est la probabilité et où se trouve t-elle par rapport au seuil ?,Données nécessaires: -score client -probabilité client Indicateurs à afficher : -Business_score -predict_proba,
2,Interprétation du score de façon simple,Quels paramètres ont contribués le plus au choix ?,Données nécessaires : Feature Importance Locale Indicateurs à afficher : -TreeExplainer de shap,
3,Principales informations descriptives du client,"-Age, sexe, Possède une voiture ?, Possède un logement ? -Quelle est la situation professionnelle du client ? -Quelle est sa situation maritale ? enfant ?","Données nécessaires : -Informations descriptives du client en cours Indicateurs à afficher : -SK_ID_CURR, DAYS_BIRTH, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY -AMT_INCOME_TOTAL -NAME_FAMILY_STATUS -CNT_CHILDREN",
4,Comparaison des infos du client par rapport à: -L'ensemble des clients Un groupe (filtres),Où se trouve ce client par rapport aux autres ?,Données nécessaires : informations descriptives de tous les clients Indicateurs à afficher : -Moyenne globale ou par groupe des informations descriptives,


In [5]:
# folder path

folder = "././Projet_07/data/source/"

In [16]:
# list files available

pprint(os.listdir(folder))

['application_test.csv',
 '.ipynb_checkpoints',
 'sample_submission.csv',
 'bureau.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'bureau_balance.csv',
 'POS_CASH_balance.csv',
 'installments_payments.csv',
 'application_train.csv',
 'previous_application.csv']


In [17]:
df = pd.read_csv(folder + 'application_train.csv')

In [31]:
list_cols = df.columns.to_list()

In [32]:
print(list_cols)

['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELE

In [75]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


#### Actions :

* Fichier global complet avec predictions à mettre en cache dans Streamlit
* MAJ des graphs pour mettre la valeur du client en lumière
* Shap à mettre à recalculer à chaque modif de client

#### Fonctions nécessaires :

* `retrieve_client_data` : récupère la ligne de data du client 
* `display_client_form` : affichage des données du client dans le form "client"
* `display_credit_form` : affichage des données du crédit dans le form "credit"
* `update_client_graphs` : mise à jour des graphs client en mettant en lumière la valeur du client
* `shap_explainer` : calcul et affichage de l'explainer du client dans le form shap
* `update_bivariate_graph` : mise à jour du graph bivarié en changeant une des valeurs

---
### 1.3 - Récupération de l'explainer SHAP :